For solving path dependencies

In [1]:
import sys
import os

current_dir = %pwd

# Detect the parent directory dynamically
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)


# DEVE CHIAMARE LOG EXPERIMENTS PER SALVARE GLI ESPERIMENTI SU UN CSV

(124, 17)
(124, 1)
(432, 17)
(432, 1)
------ Train scores: ------ 
Accuracy 	 0.75 +\- 0.11483684077855852
loss 	 0.2499041739766008 +\- 0.00016189963994249406
------ Validation scores: ------ 
Accuracy 	 0.7 +\- 0.1224744871391589
loss 	 0.2500092554794205 +\- 0.0006091545059478398
------ Test scores: ------ 
Accuracy 	 0.6936666666666667 +\- 0.14413111314972135
loss 	 0.14868601800451214 +\- 0.05655789796267408
